In [1]:
import pandas as pd
import h5py
from st_datasets.dataset import *
from tools import evaluate

result = []

f = h5py.File('result(2).h5', 'r')
df = pd.DataFrame(f['imputed_data'])
adat, n_cluster, _ = get_human_breast_cancer_data()
adat.X = df
sc.pp.highly_variable_genes(adat, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adat, target_sum=1e4)
sc.pp.log1p(adat)

OSError: Unable to open file (truncated file: eof = 1015414784, sblock->base_addr = 0, stored_eof = 1016037719)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import adjusted_rand_score

res = KMeans(n_clusters=n_cluster, random_state=0).fit(adat.X).predict(adat.X)
label = LabelEncoder().fit_transform([str(x) for x in res])
ARI = adjusted_rand_score(label, adat.obs['cluster']) * 100
ARI

/mnt/5468e/twang/.conda/envs/pyg/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


69.36789975924572

In [ ]:
adata = adat.copy()
from CCST.run import train

train(adata, 300, n_cluster)
ari, ari_pca = evaluate(adata, pred_list=['CCST', 'CCST_pca'])
result.append(ari)

===== Preprocessing Data 
calculating distance matrix, it takes a while
300 22064 2.9046866771985256


/mnt/5468e/twang/.conda/envs/pyg/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


>>> graph contains 22064 edges, 5.809373354397051 edges per node
>>> begin to train DGI, shape: (3798, 300)


100%|██████████| 5000/5000 [01:06<00:00, 74.63it/s]


Training time in seconds:  66
Shape of data to PCA: (3798, 256)
Shape of data output by PCA: (3798, 50)
PCA recover: 0.994666
>>> CCST: 79.9299
>>> CCST_pca: 79.4841


In [ ]:
adata = adat.copy()
from sklearn import metrics
from GraphST.GraphST import GraphST
from GraphST.utils import clustering

model = GraphST(adata, device='cuda')
adata = model.train()
radius = 50
clustering(adata, n_cluster, radius=radius, method='mclust', refinement=False)
obs_df = adata.obs.dropna()
ARI = metrics.adjusted_rand_score(obs_df['domain'], obs_df['cluster'])*100
result.append(ARI)

Begin to train ST data...


100%|██████████| 600/600 [00:13<00:00, 45.92it/s]


Optimization finished for ST data!


R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 5.4.10
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


In [ ]:
adata = adat.copy()
from train_SpaGCN import train_spg

train_spg(adata, n_cluster)
ari, ari_refine = evaluate(adata, pred_list=['pred', 'refined_pred'])
result.append(ari)

Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 83.64162436864945]
Run 2: l [0.01, 500.005], p [0.0, 21.58456802368164]
Run 3: l [0.01, 250.0075], p [0.0, 4.846828460693359]
Run 4: l [0.01, 125.00874999999999], p [0.0, 0.5336281061172485]
Run 5: l [62.509375, 125.00874999999999], p [0.00040018558502197266, 0.5336281061172485]
Run 6: l [93.7590625, 125.00874999999999], p [0.0820695161819458, 0.5336281061172485]
Run 7: l [109.38390625, 125.00874999999999], p [0.2545154094696045, 0.5336281061172485]
Run 8: l [117.196328125, 125.00874999999999], p [0.3819239139556885, 0.5336281061172485]
Run 9: l [121.1025390625, 125.00874999999999], p [0.4549444913864136, 0.5336281061172485]
recommended l =  123.05564453125
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  22
Initializing cluster centers with louvain, resolution =  0.6
Epoch  0
Epoch  10
Res =  0.6 Num of clusters =  22
Res 

In [ ]:
adata = adat.copy()
import STAGATE_pyG as STAGATE
from sklearn.metrics import adjusted_rand_score

STAGATE.Cal_Spatial_Net(adata, rad_cutoff=300)
adata = STAGATE.train_STAGATE(adata)
adata = STAGATE.mclust_R(adata, used_obsm='STAGATE', num_cluster=n_cluster)
obs_df = adata.obs.dropna()
ARI = adjusted_rand_score(obs_df['mclust'], obs_df['cluster'])*100
result.append(ARI)

------Calculating spatial graph...
The graph contains 22064 edges, 3798 cells.
5.8094 neighbors per cell on average.
Size of Input:  (3798, 3000)


100%|██████████| 1000/1000 [00:11<00:00, 88.35it/s]

fitting ...
  |                                                                      |   0%

  |======================================================================| 100%


In [ ]:
print(result)

[79.92986283985823, 80.8858804023149, 95.18891774936975, 70.68702659284463]
